<img src='img/ansible-logo.jpg'>

Ansible - системама управления конфигурацией (Configuration Management System).

___
## <center><a id=1>Установка ansible</a>

In [ ]:
sudo apt-get update
sudo apt-get install software-properties-common
sudo apt-add-repository ppa:ansible/ansible
sudo apt-get install ansible

___
## <center><a id=2>Конфигурация ansible</a>

- исходные директории ansible

In [ ]:
ansible --version

- ansible.cfg

- `inventory`: директория с группами хостов
- `forks`: макс кол-во хостов для запуска задания
- `sudo_user`: под каким пользователем буде ansible на хосте
- `ask_sudo_pass`: будет ли ansible запрашивать пароль для становления другим пользователем
- `ask_pass`: будет ли ansible запрашивать пароль (если не настроена авторизация по ключу)
- `gathering`: будет ли ansible собирать факты об удаленном хосте
- `roles_path`: директория с заданными ролями для хостов
- `host_key_checking`: опция не проверять ключ хоста
- `remote_user`: пользователь под которым ansible подключится к хосту и выполнит задание


- hosts (inventory): написание групп хостов для выполнения задания

[название группы хостов]
Хост1
Хост2
...

___
## <center><a id=2>Playbook</a>

1. Вывести объем ОП и кол-во вирт. процессоров хоста

-hosts: группа хостов для выполнения задания  
gather_facts: собирать факты о машине  
tasks: задание
- -name: имя задания
- become: использование sudo для выполнения команд
- template: шаблон для выполнения
    - src: расположение шаблона
    - dest: файл назначений


In [ ]:
---
- hosts: test
  gather_facts: yes
  tasks:
    - name: edit motd file
      become: true
      template:
        src: /etc/ansible/playbooks/templates/motd.j2
        dest: /etc/motd

motd.j2: Шаблон
- {{переменная ansible, которая собирает данные о системе}}
    - {Объем ОП}
    - {Кол-во вирт. процессоров}

In [ ]:
There is {{ansible_facts.facter_memory.system.total}} memory total
and {{ansible_facts.processor_vcpus}} vCPU(s)

2. Выдать ошибку, если используется python версии 2

In [ ]:
---
- hosts: test
  gather_facts: yes
  tasks:
    - name: create warning in file
      become: true
      template:
        src: /etc/ansible/playbooks/templates/warn.j2
        dest: /tmp/warn_py

warn.j2: Шаблон  

- {если python версии 2}  
    - Сообщение об ошибке  
    - {или пустой файл}  

In [ ]:
{% if ansible_facts.python.version.major == 2 %}
Warning: using depricated python version 2!
{% endif %}

3. Развернуть httpd-сервер со входом по логуину и паролю 

In [ ]:
---
- hosts: test
  gather_facts: no
  tasks:
  - name: install httpd
    become: true
    apt:
       name: httpd
       state: install
  - name: remove conf.d directory
    become: true
    file:
        path: /etc/httpd/conf.d
        state: absent
  - name: create conf.d
    become: true
    file:
        path: /etc/httpd/conf.d
        state: directory
  - name: add new virt host config
    become: true
    copy:
        src: /etc/ansible/playbooks/files/vhost.conf
        dest: /etc/httpd/conf.d/host1.conf
        remote_src: yes
    notify: restart_httpd
  - name: create htpasswd
    become: true
    command: 'htpasswd -b -c /etc/httpd/.htpasswd user1 q1w2e3'
  - name: create content
    become: true
    copy:
        src: /etc/ansible/playbooks/files/content.html
        dest: /var/www/html/index.html
        remote_src: yes
  handlers:
  - name: restart httpd
    become: true
    listen: restart_httpd
    service:
      name: httpd
      state: restarted

Проверка синтаксиса playbook

In [ ]:
ansible-playbook --syntax-check <файл >

___
## <center><a id=3>Шифрование данных</a>

Зашифровка и создание файла с секретным содержимым

In [ ]:
ansible-vault create <файл>

Зашифровка  файла с секретным содержимым


In [ ]:
ansible-vault encrypt <файл>

Вывод содержимого файла

In [ ]:
ansible-vault view <файл>

Расшифровка файла с секретным содержимым

In [ ]:
ansible-vault decrypt  <файл>

In [ ]:
ansible-vault create --vault-password-file <файл с паролем > <файл >